In [2]:
import pandas as pd
import numpy as np


In [3]:
file = "GAsP_Multiallelic_Y_SNP_Microhap_MAF_RMP.csv"  # enter the file name

### Naming Scheme MH1,MH2, for all SNPs

In [4]:
df = pd.read_csv(file)
df.head(3)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,MAF,...,GA001886,GA001887,GA001888,GA001890,GA001891,GA001892,GA001893,GA001894,GA001895,MajorAF.1
0,Y,2984956,.,A,"*,C",52936.70,PASS,"GT1168=2:3,1:112,0:393,.:655",GT:AD:DP:GQ:PL,0.250000,...,"0:3,0,0:3:42:0,42,42","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.",".:0,0,0:.:.:.","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",0.750000
1,Y,2984958,.,A,"C,*",4390.69,PASS,"GT1168=2:11,1:2,0:529,.:621",GT:AD:DP:GQ:PL,0.083333,...,"0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.","0:1,0,0:1:37:0,37,37","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",0.916667
2,Y,2984960,.,A,"C,*",3267.29,PASS,"GT1168=1:7,0:536,.:620",GT:AD:DP:GQ:PL,0.000000,...,"0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.","0:1,0,0:1:42:0,42,42","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",1.000000


In [32]:
MAX_DISTANCE = 200                                 # Maximum distance between SNPs to be considered in the same cluster
SNP_NUM = 3                                        # Minimum number of SNPs required to form a cluster

snp_cluster_list = []                              # List to hold SNP clusters
snp_pos_list = df['POS'].tolist()                  # Convert SNP positions to a list

temp_cluster_list = [snp_pos_list[0]]              # Temporary list to hold current cluster of SNPs
temp_group_id_list = []                            # Temporary list to hold group IDs for current cluster
group_id = 1                                       # Initialize group ID for clusters

# Assign group IDs for all SNPs, default to 'UNGROUPED'
group_ids = ['UNGROUPED'] * len(snp_pos_list)


# MAX_DISTANCE = 200
# SNP_NUM = 3
# snp_cluster_list = []
# snp_pos_list = df['POS'].tolist()
# temp_cluster_list = [snp_pos_list[0]]
# temp_group_id_list = []
# group_id = 1

# # Assign group IDs for all SNPs, default to 'UNGROUPED'
# group_ids = ['UNGROUPED'] * len(snp_pos_list)

# for i in range(1, len(snp_pos_list)):
#     if (snp_pos_list[0] - snp_pos_list[i-1]) <= MAX_DISTANCE:
#         temp_cluster_list.append(snp_pos_list[i])
#     else:
#         if len(temp_cluster_list) >= SNP_NUM:
#             snp_cluster_list.extend(temp_cluster_list)
#             for idx in range(i - len(temp_cluster_list), i):
#                 group_ids[idx] = f"MH{group_id}"
#             group_id += 1
#         temp_cluster_list = [snp_pos_list[i]]

# # If at the end of the loop the size of temp_cluster_list is more than 3
# if len(temp_cluster_list) >= SNP_NUM:
#     snp_cluster_list.extend(temp_cluster_list)
#     for idx in range(len(snp_pos_list) - len(temp_cluster_list), len(snp_pos_list)):
#         group_ids[idx] = f"MH{group_id}"
#     group_id += 1

# print(f"snp_pos_list length: {len(snp_pos_list)}")
# print(f"group_ids length: {len(group_ids)}")


# Iterate through the SNP positions starting from the second SNP
for i in range(1, len(snp_pos_list)):
    if abs(snp_pos_list[i] - snp_pos_list[i - 1]) <= MAX_DISTANCE:  # Considering the absolute difference between the current SNP and the last SNP in the cluster
        temp_cluster_list.append(snp_pos_list[i])
    else:
        if len(temp_cluster_list) >= SNP_NUM:
            snp_cluster_list.append(temp_cluster_list)  # Use append() to add the whole cluster as one element or extend() elementwise
            for idx in range(i - len(temp_cluster_list), i):
                group_ids[idx] = f"MH{group_id}"
            group_id += 1
        else:
            snp_cluster_list.extend(temp_cluster_list)  # Use extend() to add individual SNP positions since they are not part of a valid cluster
        temp_cluster_list = [snp_pos_list[i]]

# If at the end of the loop the temp_cluster_list is not empty
if len(temp_cluster_list):
    if len(temp_cluster_list) >= SNP_NUM:
        snp_cluster_list.append(temp_cluster_list)
        for idx in range(len(snp_pos_list) - len(temp_cluster_list), len(snp_pos_list)):
            group_ids[idx] = f"MH{group_id}"
        group_id += 1
    else:
        snp_cluster_list.extend(temp_cluster_list)

# Verification of results

print(f"snp_pos_list length: {len(snp_pos_list)}")
print(f"group_ids length: {len(group_ids)}")


snp_pos_list length: 321
group_ids length: 321


In [33]:
df.columns

In [33]:
group_series = pd.Series({'GroupID': group_ids})
df["GroupID"] = group_ids
df["POS"] = snp_pos_list
cols = ['GroupID'] + [col for col in df.columns if col != 'GroupID']
df = df[cols]

In [34]:
df.head()

,GroupID,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,...,GA001886,GA001887,GA001888,GA001890,GA001891,GA001892,GA001893,GA001894,GA001895,MajorAF.1
0,MH1,Y,2984956,.,A,"*,C",52936.70,PASS,"GT1168=2:3,1:112,0:393,.:655",GT:AD:DP:GQ:PL,...,"0:3,0,0:3:42:0,42,42","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.",".:0,0,0:.:.:.","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",0.750000
1,MH1,Y,2984958,.,A,"C,*",4390.69,PASS,"GT1168=2:11,1:2,0:529,.:621",GT:AD:DP:GQ:PL,...,"0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.","0:1,0,0:1:37:0,37,37","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",0.916667
2,MH1,Y,2984960,.,A,"C,*",3267.29,PASS,"GT1168=1:7,0:536,.:620",GT:AD:DP:GQ:PL,...,"0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,111,111","0:1,0,0:1:45:0,45,45",".:0,0,0:.:.:.","0:1,0,0:1:42:0,42,42","0:2,0,0:2:90:0,90,90","0:1,0,0:1:45:0,45,45","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.",1.000000
3,MH2,Y,3320873,.,T,"G,A",2741.70,PASS,"GT1168=2:1,1:3,0:626,.:533",GT:AD:DP:GQ:PL,...,"0:4,0,0:4:99:0,135,135","0:3,0,0:3:99:0,99,99","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.","0:4,0,0:4:99:0,135,135","0:6,0,0:6:99:0,180,180","0:5,0,0:5:99:0,180,180","0:4,0,0:4:99:0,135,135",".:0,0,0:.:.:.",0.500000
4,MH2,Y,3320910,.,C,"*,T",4687.68,PASS,"GT1168=2:2,1:1,0:627,.:533",GT:AD:DP:GQ:PL,...,"0:4,0,0:4:99:0,135,135","0:3,0,0:3:99:0,99,99","0:3,0,0:3:99:0,123,123",".:0,0,0:.:.:.","0:4,0,0:4:99:0,135,135","0:6,0,0:6:99:0,180,180","0:5,0,0:5:99:0,180,180","0:4,0,0:4:99:0,135,135",".:0,0,0:.:.:.",0.666667


In [8]:
df.groupby('GroupID')[df.columns].value_counts()

GroupID  CHROM  POS       ID           REF  ALT  QUAL      FILTER  INFO                          FORMAT          MAF       MajorAF   RMP       GA000432       GA000433                  GA000434       GA000435       GA000436       GA000437       GA000438       GA000439                    GA000440                  GA000441                    GA000442                    GA000443                    GA000444                   GA000445                  GA000446                    GA000447                    GA000448                   GA000449                   GA000450                   GA000451                   GA000452                    GA000453                   GA000454                  GA000455                    GA000456                  GA000457       GA000459       GA000460       GA000461       GA000462       GA000463       GA000464       GA000465       GA000466       GA000467       GA000468                   GA000469                  GA000470                  GA000471              

In [36]:
# save the file
file_save = "GAsP_Multiallelic_output.csv"  # enter the save file name
df.to_csv(file_save, index=False)